In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#!pip install pyyaml==5.1 pycocotools>=2.0.1
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version

1.6.0+cu101 True
gcc (Ubuntu 7.5.0-3ubuntu1~18.04) 7.5.0
Copyright (C) 2017 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
assert torch.__version__.startswith("1.6")
#!pip install detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.6/index.html

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
%cd drive/My Drive/cyc3600

/content/drive/My Drive/cyc3600


In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg
import os

cfg = get_cfg()
#cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_101_FPN_3x.yaml"))
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_R_101_DC5_3x.yaml"))
# cfg.merge_from_file(
#     "./detectron2_repo/configs/COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x.yaml"
# )
#cfg.DATASETS.TRAIN = ("cyclone_ds_train",)
#cfg.DATASETS.TEST = ()  # no metrics implemented for this dataset
cfg.DATALOADER.NUM_WORKERS = 2
#cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"  # initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 1
cfg.SOLVER.BASE_LR = 0.002
cfg.SOLVER.MAX_ITER = (
    2000
)  # 300 iterations seems good enough, but you can certainly train longer
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = (
    1280
)  # faster, and good enough for this toy dataset
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4  # 3 classes (data, fig, hazelnut)
cfg.INPUT.MAX_SIZE_TEST = 3600
cfg.INPUT.MAX_SIZE_TRAIN = 3600
cfg.INPUT.MIN_SIZE_TEST = 1201
cfg.INPUT.MIN_SIZE_TRAIN = 1201
cfg.MODEL.ANCHOR_GENERATOR.SIZES = [[60,70,80,90]]

In [ ]:
from detectron2.modeling import build_model
model = build_model(cfg)

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer
#DetectionCheckpointer(model).load('Mod-DC5-R101-b80/model_final.pth')
DetectionCheckpointer(model).load('b80-R101-DC5-3x/model_final.pth')


{'iteration': 1999,
 'optimizer': {'param_groups': [{'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [0],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [1],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [2],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [3],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [4],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'initial_lr': 0.002,
    'lr': 0.002,
    'momentum': 0.9,
    'nesterov': False,
    'params': [5],
    'weight_decay': 0.0001},
   {'dampening': 0,
    'in

In [ ]:
import torch
from PIL import Image

pred = DefaultPredictor(cfg)


# image = Image.open('test/ssd_20201006_00.png')
# #X = torch.from_numpy(np.array(image))

# X = torch.Tensor(np.array(image))

# print(type(X))
# #exit()

# inputs = [{"image": X}]

# outputs = model(inputs)

[10/15 14:16:38 d2.checkpoint.c2_model_loading]: Remapping C2 weights ......
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv1.norm.bias                      loaded from res2_0_branch2a_bn_beta           of shape (64,)
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv1.norm.running_mean              loaded from res2_0_branch2a_bn_running_mean   of shape (64,)
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv1.norm.running_var               loaded from res2_0_branch2a_bn_running_var    of shape (64,)
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv1.norm.weight                    loaded from res2_0_branch2a_bn_gamma          of shape (64,)
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv1.weight                         loaded from res2_0_branch2a_w                 of shape (64, 64, 1, 1)
[10/15 14:16:38 d2.checkpoint.c2_model_loading]: backbone.res2.0.conv2.norm.bias                   

In [ ]:
import glob

flist = glob.glob('test/*')
for each in flist:
  print(each)
  input = cv2.imread(each)
  out = pred(input)
  print(out)

test/ssd_20201006_00.png
{'instances': Instances(num_instances=100, image_height=1201, image_width=3600, fields=[pred_boxes: Boxes(tensor([[2446.6797,  458.2839, 2522.6826,  623.9603],
        [2454.1521,  474.5798, 2474.1711,  633.8996],
        [2394.2214,  496.8755, 2425.2048,  635.3854],
        [3469.2305,  324.9399, 3513.0166,  425.0599],
        [2304.0999,  380.9690, 2449.2742,  516.6835],
        [2340.1956,  473.5372, 2429.7834,  636.4015],
        [2461.6414,  473.0238, 2476.0886,  560.9923],
        [1493.8372,  526.9988, 1564.9961,  564.4794],
        [2439.8005,  449.8811, 2482.0793,  644.6406],
        [3452.5115,  323.9926, 3518.8923,  385.4960],
        [3461.5522,  326.6824, 3486.9551,  367.0848],
        [1455.2877,  549.8283, 1568.0538,  580.3880],
        [ 736.7274,  158.1056,  799.5929,  263.5876],
        [1887.8060,  729.3534, 1925.9772,  854.1661],
        [3415.8125,  331.0149, 3469.4404,  400.8333],
        [1414.6158,  404.8216, 1439.1959,  496.2357],
     